In [14]:
import pandas as pd
import numpy as np


import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()

import multiprocessing

import pyspark.sql.functions as F

In [15]:

nprocs = multiprocessing.cpu_count()

spark = (pyspark.sql.SparkSession.builder
 .master('local')
 .config('spark.jars.packages', 'mysql:mysql-connector-java:8.0.16')
 .config('spark.driver.memory', '4G')
 .config('spark.driver.cores', nprocs)
 .config('spark.sql.shuffle.partitions', nprocs)
 .appName('MySparkApplication')
 .getOrCreate())

## Exercises
Using the repo setup directions, setup a new local and remote repository named spark-exercises. The local version of your repo should live inside of ~/codeup-data-science. This repo should be named spark-exercises

Save this work in your spark-exercises repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named spark101 for this exercise.

### Create a spark data frame that contains your favorite programming languages.

a. The name of the column should be language\
b. View the schema of the dataframe\
c. Output the shape of the dataframe\
d. Show the first 5 records in the dataframe

In [16]:
lang_df = spark.createDataFrame(
    pd.DataFrame(
        {
            "language": [
                "python",
                "sql",
                "javascript"
            ]
        }
    )
)

lang_df.show(truncate=False)

+----------+
|language  |
+----------+
|python    |
|sql       |
|javascript|
+----------+



In [17]:
lang_df.printSchema()



root
 |-- language: string (nullable = true)



In [18]:
print(lang_df.count(), len(lang_df.columns))

3 1


In [19]:
lang_df.show(5)

+----------+
|  language|
+----------+
|    python|
|       sql|
|javascript|
+----------+



### Load the mpg dataset as a spark dataframe.

a. Create 1 column of output that contains a message like the one below:

    ```The 1999 audi a4 has a 4 cylinder engine.``` 
     For each vehicle.


b. Transform the trans column so that it only contains either manual or auto.

In [20]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))

In [21]:
mpg.show(3)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 3 rows



In [34]:
# Create a new column in a copy of our Spark DataFrame.
# Note: This does not change the original DataFrame.
from pyspark.sql.functions import concat, lit

(
    mpg
    .withColumn(
        'full_description',
        concat(
            lit('The '), 
            mpg['year'],
            lit(' '),  # Add spaces or separators as needed
            mpg['manufacturer'],
            lit(' '),
            mpg['model'], 
            lit(' has a '), 
            mpg['cyl'],
            lit(' cylinder engine.')
        )
    )
    .show(5)
)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|    full_description|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|The 1999 audi a4 ...|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|The 1999 audi a4 ...|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|The 2008 audi a4 ...|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|The 2008 audi a4 ...|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|The 1999 audi a4 ...|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------+
only showing top 5 rows



In [36]:
from pyspark.sql.functions import when, lit, concat, col

(
    mpg
    .withColumn(
        'full_description',
        concat(
            lit('The '), 
            mpg['year'],
            lit(' '),  
            mpg['manufacturer'],
            lit(' '),
            mpg['model'], 
            lit(' has a '),
            when(col('trans').startswith('a'), lit('automatic'))
            .otherwise(lit('manual')), 
            lit(' transmission.')
        )
    )
    .show(5)
)


+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|    full_description|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|The 1999 audi a4 ...|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|The 1999 audi a4 ...|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|The 2008 audi a4 ...|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|The 2008 audi a4 ...|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|The 1999 audi a4 ...|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------+
only showing top 5 rows



### Load the tips dataset as a spark dataframe.

a. What percentage of observations are smokers?\
b. Create a column that contains the tip percentage\
c. Calculate the average tip percentage for each combination of sex and smoker.

In [53]:
from pydataset import data

tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [55]:
tips.groupby('smoker').count().show()

+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



In [ ]:
sdf.groupBy('class').count().show()

### Use the seattle weather dataset referenced in the lesson to answer the questions below.

* Convert the temperatures to fahrenheit.
* Which month has the most rain, on average?
* Which year was the windiest?
* What is the most frequent type of weather in January?
* What is the average high and low temperature on sunny days in July in 2013 and 2014?
* What percentage of days were rainy in q3 of 2015?
* For each year, find what percentage of days it rained (had non-zero precipitation).